In [1]:
import pandas as pd

# Carga el archivo Excel
df = pd.read_excel('VentaViviendas.xlsx')  # Asegúrate de colocar el archivo en el mismo directorio o proporcionar la ruta completa.

# Calcula la mediana de la columna "lat"
mediana_lat = df['lat'].median()
print(f"La mediana de 'lat' es: {mediana_lat}")


La mediana de 'lat' es: 47.5745


In [2]:
valores_unicos = df['condition'].nunique()
print(f"El número de valores únicos en 'condition' es: {valores_unicos}")


El número de valores únicos en 'condition' es: 5


In [3]:
cantidad_categoria_0 = df[df['waterfront'] == 0].shape[0]
print(f"La cantidad de instancias que pertenecen a la categoría 0 de 'waterfront' es: {cantidad_categoria_0}")


La cantidad de instancias que pertenecen a la categoría 0 de 'waterfront' es: 4852


In [4]:
# Calcula el porcentaje de valores faltantes en la columna 'waterfront'
porcentaje_faltantes = (df['waterfront'].isnull().sum() / len(df)) * 100
print(f"El porcentaje de filas faltantes en 'waterfront' es: {porcentaje_faltantes:.2f}%")


El porcentaje de filas faltantes en 'waterfront' es: 0.00%


In [22]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import mean_absolute_percentage_error as MAPE

# Cargar el archivo Excel
df = pd.read_excel('IPI_Esp.xlsx')  # Asegúrate de que el archivo esté en el mismo directorio o proporciona la ruta completa.

# Limpiar la columna 'Date' para eliminar espacios adicionales
df['Date'] = df['Date'].str.strip()  # Eliminar espacios antes y después de los valores

# Reemplazar el formato '1975M01' a '1975-01'
df['Date'] = df['Date'].str.replace('M', '-')  # Reemplazar 'M' por '-'

# Convertir la columna 'Date' a formato datetime
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m', errors='coerce')  # Manejo de errores para filas mal formateadas

# Verificar si hay fechas no convertidas
if df['Date'].isna().sum() > 0:
    print("Advertencia: Existen fechas no convertidas correctamente.")
    print(df[df['Date'].isna()])  # Mostrar las filas problemáticas

# Dividir el conjunto de datos en ventanas de entrenamiento y prueba
train = df[df['Date'] <= '2017-12-31']  # Datos hasta el 31 de diciembre de 2017 (entrenamiento)
test = df[df['Date'] > '2017-12-31']    # Datos desde el 1 de enero de 2018 (prueba)

# Ajustar el modelo aditivo de Holt-Winters
model = ExponentialSmoothing(
    train['IPI Nacional'],  # Cambia 'IPI Nacional' si la columna tiene otro nombre
    seasonal='add',         # Estacionalidad aditiva
    seasonal_periods=12,    # Periodo estacional (mensual)
    trend='add'             # Tendencia aditiva
).fit()

# Predicción en la ventana de prueba
forecast = model.forecast(len(test))

# Cálculo del MAPE
mape_value = MAPE(test['IPI Nacional'], forecast)  # Cambia 'IPI Nacional' si la columna tiene otro nombre
print(f"MAPE: {mape_value:.2f}")

# Test de Ljung-Box para los residuos
ljung_box_pvalue = acorr_ljungbox(model.resid, lags=[12], return_df=True)['lb_pvalue'].iloc[0]
print(f"Valor p de Ljung-Box: {ljung_box_pvalue:.3e}")



MAPE: 0.03
Valor p de Ljung-Box: 1.623e-22


In [23]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import mean_absolute_percentage_error as MAPE

# Cargar el archivo Excel
df = pd.read_excel('IPI_Esp.xlsx')  # Asegúrate de que el archivo esté en el mismo directorio o proporciona la ruta completa.

# Limpiar la columna 'Date' para eliminar espacios adicionales
df['Date'] = df['Date'].str.strip()  # Eliminar espacios antes y después de los valores

# Reemplazar el formato '1975M01' a '1975-01'
df['Date'] = df['Date'].str.replace('M', '-')  # Reemplazar 'M' por '-'

# Convertir la columna 'Date' a formato datetime
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m', errors='coerce')

# Verificar si hay fechas no convertidas
if df['Date'].isna().sum() > 0:
    print("Advertencia: Existen fechas no convertidas correctamente.")
    print(df[df['Date'].isna()])  # Mostrar las filas problemáticas

# Configurar la columna 'Date' como índice
df.set_index('Date', inplace=True)

# Dividir el conjunto de datos en ventanas de entrenamiento y prueba
train = df[df.index <= '2017-12-31']  # Datos hasta el 31 de diciembre de 2017 (entrenamiento)
test = df[df.index > '2017-12-31']    # Datos desde el 1 de enero de 2018 (prueba)

# Ajustar el modelo SARIMAX(1,1,1)(1,1,1)[12]
model = SARIMAX(
    train['IPI Nacional'],  # Cambia 'IPI Nacional' si la columna tiene otro nombre
    order=(1, 1, 1),         # Parámetros SARIMA (p, d, q)
    seasonal_order=(1, 1, 1, 12)  # Parámetros estacionales (P, D, Q, s)
).fit(disp=False)

# Generar predicciones
forecast = model.forecast(steps=len(test))

# Cálculo del MAPE
mape_value = MAPE(test['IPI Nacional'], forecast)  # Cambia 'IPI Nacional' si la columna tiene otro nombre
print(f"MAPE: {mape_value:.2f}")

# Test de Ljung-Box para los residuos
ljung_box_pvalue = acorr_ljungbox(model.resid, lags=[12], return_df=True)['lb_pvalue'].iloc[0]
print(f"Valor p de Ljung-Box: {ljung_box_pvalue:.4f}")


C:\Users\jarpl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\jarpl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


MAPE: 0.03
Valor p de Ljung-Box: 0.0000


In [32]:
import pandas as pd

# Cargar el archivo Excel llamado VentaViviendas.xlsx
df = pd.read_excel('VentaViviendas.xlsx')  # Asegúrate de que el archivo esté en la misma carpeta o proporciona la ruta completa.

# Verificar las columnas del DataFrame para confirmar que 'yr_renovated' está presente
print("Columnas del archivo:", df.columns)

# Limpiar espacios adicionales en los nombres de las columnas (si es necesario)
df.columns = df.columns.str.strip()

# Verificar los valores únicos en 'yr_renovated' para asegurar consistencia
print("Valores únicos en la columna 'yr_renovated':", df['yr_renovated'].unique())

# Contar las instancias donde 'yr_renovated' es igual a 0
cantidad_cero = (df['yr_renovated'] == 0).sum()
print(f"La cantidad de viviendas con 'yr_renovated' igual a 0 es: {cantidad_cero}")


Columnas del archivo: Index(['year', 'month', 'price', 'Luxury', 'bedrooms', 'bathrooms',
       'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition',
       'sqft_above', 'basement', 'yr_built', 'yr_renovated', 'lat', 'long'],
      dtype='object')
Valores únicos en la columna 'yr_renovated': [   0 2006 1983 1992 2001 2003 2009 2012 1989 1990 2007 1984 1955 2000
 1995 2015 2014 2004 1978 1994 1977 1991 2002 1968 1965 1999 1987 1986
 1996 1993 1979 1998 1985 1973 1970 2011 1988 2005 1976 1980 2010 1982
 1971 2013 1972 1974 1967 2008 1960 1957 1953 1945 1964 1969 1997 1963
 1950 1956]
La cantidad de viviendas con 'yr_renovated' igual a 0 es: 4784


In [35]:
import pandas as pd
import statsmodels.formula.api as smf

# Cargar el archivo Excel
df = pd.read_excel('VentaViviendas.xlsx')  # Asegúrate de que el archivo esté en el mismo directorio o proporciona la ruta completa.

# Verificar las columnas del archivo
print("Columnas del archivo:", df.columns)

# Ajustar el modelo de regresión lineal con 'price' como precio
modelo = smf.ols('price ~ lat', data=df).fit()

# Obtener los resultados del modelo
print(modelo.summary())

# Imprimir el valor de R²
r2 = modelo.rsquared
print(f"El valor de R² es: {r2:.3f}")


Columnas del archivo: Index(['year', 'month', 'price', 'Luxury', 'bedrooms', 'bathrooms',
       'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition',
       'sqft_above', 'basement', 'yr_built', 'yr_renovated', 'lat', 'long'],
      dtype='object')
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     498.6
Date:                Wed, 19 Mar 2025   Prob (F-statistic):          2.30e-105
Time:                        20:17:06   Log-Likelihood:                -71003.
No. Observations:                5000   AIC:                         1.420e+05
Df Residuals:                    4998   BIC:                         1.420e+05
Df Model:                           1                                         
Covariance Type:     

In [36]:
import pandas as pd

# Cargar el archivo Excel
df = pd.read_excel('VentaViviendas.xlsx')  # Asegúrate de que el archivo esté en el mismo directorio o proporciona la ruta completa.

# Calcular la media de 'sqft_living'
media_sqft_living = df['sqft_living'].mean()
print(f"La media de sqft_living es: {media_sqft_living:.3f}")


La media de sqft_living es: 2077.382


In [37]:
import pandas as pd

# Cargar el archivo Excel
df = pd.read_excel('VentaViviendas.xlsx')  # Asegúrate de que el archivo esté en el mismo directorio o proporciona la ruta completa.

# Calcular la desviación típica de 'price'
desviacion_price = df['price'].std()
print(f"La desviación típica de price es: {desviacion_price:.1f}")


La desviación típica de price es: 372986.9


In [38]:
import pandas as pd

# Cargar el archivo Excel
df = pd.read_excel('VentaViviendas.xlsx')  # Asegúrate de que el archivo esté en el mismo directorio o proporciona la ruta completa.

# Calcular el número de valores únicos en la columna 'bedrooms'
valores_unicos = df['bedrooms'].nunique()
print(f"El número de valores únicos en 'bedrooms' es: {valores_unicos}")


El número de valores únicos en 'bedrooms' es: 33


In [42]:
import pandas as pd

# Cargar el archivo Excel
df = pd.read_excel('VentaViviendas.xlsx')  # Asegúrate de que el archivo esté en el mismo directorio o proporciona la ruta completa.

# Identificar valores considerados "missing" (nulos, negativos o igual a cero)
missing_sqft_lot = df['sqft_lot'].isnull().sum()  # Valores nulos
missing_negativos_cero = (df['sqft_lot'] <= 0).sum()  # Valores negativos o igual a cero

# Total de valores missing
total_missing = missing_sqft_lot + missing_negativos_cero

# Calcular el porcentaje de valores "missing"
total_filas = len(df)
porcentaje_missing = (total_missing / total_filas) * 100

print(f"El porcentaje de valores 'missing' en 'sqft_lot' (nulos, negativos o cero) es: {porcentaje_missing:.2f}")


El porcentaje de valores 'missing' en 'sqft_lot' (nulos, negativos o cero) es: 7.64


In [43]:
import pandas as pd
import statsmodels.formula.api as smf

# Cargar el archivo Excel
df = pd.read_excel('VentaViviendas.xlsx')  # Asegúrate de que el archivo esté en el mismo directorio o proporciona la ruta completa.

# Ajustar el modelo de regresión lineal
modelo = smf.ols('price ~ sqft_living', data=df).fit()

# Imprimir el resumen del modelo para ver el parámetro estimado
print(modelo.summary())

# Obtener el parámetro estimado para 'sqft_living'
coef_sqft_living = modelo.params['sqft_living']
print(f"El parámetro estimado para el predictor 'sqft_living' es: {coef_sqft_living:.4f}")


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                     5252.
Date:                Wed, 19 Mar 2025   Prob (F-statistic):               0.00
Time:                        20:32:50   Log-Likelihood:                -69445.
No. Observations:                5000   AIC:                         1.389e+05
Df Residuals:                    4998   BIC:                         1.389e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -6.105e+04   9127.245     -6.689      

In [44]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

# Cargar el archivo Excel
df = pd.read_excel('VentaViviendas.xlsx')  # Asegúrate de que el archivo esté en el mismo directorio o proporciona la ruta completa.

# Ajustar el modelo de regresión logística
modelo = smf.logit('Luxury ~ sqft_living', data=df).fit()

# Imprimir el resumen del modelo
print(modelo.summary())

# Calcular el Odds Ratio para 'sqft_living'
odds_ratio = np.exp(modelo.params['sqft_living'])
print(f"El Odds Ratio estimado para 'sqft_living' es: {odds_ratio:.6f}")


Optimization terminated successfully.
         Current function value: 0.500003
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 Luxury   No. Observations:                 5000
Model:                          Logit   Df Residuals:                     4998
Method:                           MLE   Df Model:                            1
Date:                Wed, 19 Mar 2025   Pseudo R-squ.:                  0.2660
Time:                        20:33:55   Log-Likelihood:                -2500.0
converged:                       True   LL-Null:                       -3405.9
Covariance Type:            nonrobust   LLR p-value:                     0.000
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -4.1750      0.121    -34.420      0.000      -4.413      -3.937
sqft_living     0.0019   5

In [47]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import mean_absolute_percentage_error as MAPE

# Cargar el archivo Excel
df = pd.read_excel('IPI_Esp.xlsx')  # Asegúrate de que el archivo esté disponible y su ruta sea correcta.

# Limpiar espacios adicionales en la columna 'Date'
df['Date'] = df['Date'].str.strip()

# Convertir la columna 'Date' al formato datetime manejando el formato '1975M01'
df['Date'] = pd.to_datetime(df['Date'], format='%YM%m', errors='coerce')

# Verificar si hay fechas no convertidas
print("Fechas no convertidas:", df[df['Date'].isnull()])

# Eliminar filas con fechas no convertidas (si existen)
df = df.dropna(subset=['Date'])

# Establecer la columna 'Date' como índice
df.set_index('Date', inplace=True)

# Dividir los datos en ventana de entrenamiento y prueba
train = df[df.index <= '2017-12-31']
test = df[df.index > '2018-01-01']

# Ajustar el modelo SARIMAX(1,1,0)(1,1,0)[12]
model = SARIMAX(
    train['IPI Nacional'],  # Usando la columna 'IPI Nacional'
    order=(1, 1, 0),
    seasonal_order=(1, 1, 0, 12)
).fit(disp=False)

# Generar predicciones para el conjunto de prueba
forecast = model.forecast(steps=len(test))

# Calcular el MAPE
mape_value = MAPE(test['IPI Nacional'], forecast)
print(f"MAPE: {mape_value:.2f}")

# Prueba de Ljung-Box para los residuos
ljung_box_pvalue = acorr_ljungbox(model.resid, lags=[12], return_df=True)['lb_pvalue'].iloc[0]
print(f"Ljung-Box p-value: {ljung_box_pvalue:.6f}")



Fechas no convertidas: Empty DataFrame
Columns: [Date, IPI Nacional]
Index: []


C:\Users\jarpl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\jarpl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


MAPE: 0.10
Ljung-Box p-value: 0.000000
